In [124]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import seaborn as sns
import warnings, random
warnings.filterwarnings(action='ignore')

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

from sklearn.cluster import KMeans
from catboost import CatBoostRegressor



In [125]:


#pip install catboost

In [126]:
data = pd.read_csv('train.csv', encoding = 'cp949')
data_2 = pd.read_csv('test.csv', encoding = 'cp949')

In [127]:
#직업유형 결측치 '무직'으로 대체
data = data.fillna('nojob')
data_2 = data_2.fillna('nojob')

In [128]:
data.income_type.unique()

array(['Commercial associate', 'Working', 'State servant', 'Pensioner',
       'Student'], dtype=object)

In [129]:
#occyp_type은 pdf설명에 없어서
data['occyp_type'].value_counts()

nojob                    8171
Laborers                 4512
Core staff               2646
Sales staff              2539
Managers                 2167
Drivers                  1575
High skill tech staff    1040
Accountants               902
Medicine staff            864
Cooking staff             457
Security staff            424
Cleaning staff            403
Private service staff     243
Low-skill Laborers        127
Waiters/barmen staff      124
Secretaries                97
Realty agents              63
HR staff                   62
IT staff                   41
Name: occyp_type, dtype: int64

In [130]:
#소수 셋째자리까지만
pd.set_option('display.float_format', '{:.3f}'.format)
data.describe()

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit
count,26457.000,26457.000,26457.000,26457.000,26457.000,26457.000,26457.000,26457.000,26457.000,26457.000,26457.000,26457.000
mean,13228.000,0.429,187306.524,-15958.054,59068.751,1.000,0.225,0.294,0.091,2.197,-26.123,1.520
std,7637.622,0.747,101878.368,4201.589,137475.428,0.000,0.417,0.456,0.288,0.917,16.560,0.702
min,0.000,0.000,27000.000,-25152.000,-15713.000,1.000,0.000,0.000,0.000,1.000,-60.000,0.000
25%,6614.000,0.000,121500.000,-19431.000,-3153.000,1.000,0.000,0.000,0.000,2.000,-39.000,1.000
50%,13228.000,0.000,157500.000,-15547.000,-1539.000,1.000,0.000,0.000,0.000,2.000,-24.000,2.000
75%,19842.000,1.000,225000.000,-12446.000,-407.000,1.000,0.000,1.000,0.000,3.000,-12.000,2.000
max,26456.000,19.000,1575000.000,-7705.000,365243.000,1.000,1.000,1.000,1.000,20.000,0.000,2.000


In [131]:
pd.set_option('display.float_format', '{:.3f}'.format)
data_2.describe()

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month
count,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000
mean,31456.500,0.435,185043.318,-16020.466,59776.690,1.000,0.228,0.296,0.086,2.203,-26.272
std,2886.896,0.729,101539.790,4197.673,138121.225,0.000,0.419,0.457,0.280,0.898,16.349
min,26457.000,0.000,27000.000,-25152.000,-15661.000,1.000,0.000,0.000,0.000,1.000,-60.000
25%,28956.750,0.000,121500.000,-19483.250,-3153.000,1.000,0.000,0.000,0.000,2.000,-39.000
50%,31456.500,0.000,157500.000,-15606.000,-1577.000,1.000,0.000,0.000,0.000,2.000,-25.000
75%,33956.250,1.000,225000.000,-12539.000,-410.000,1.000,0.000,1.000,0.000,3.000,-12.000
max,36456.000,5.000,1575000.000,-7489.000,365243.000,1.000,1.000,1.000,1.000,7.000,0.000


In [132]:
#child_num(자녀수):  최대 19명, 평균 1명도 안됨
#DAYS_EMPLOYED: 최대  365243-> 양수값은 고용되지 않은 상태, 너무 큰 값이라 이상함
#FLAG_MOVAL: 평균이 1-> 모든 사람이 휴대폰 소지
#가족규모: 최대 20명, 최소 1명

In [133]:
#양수값들을 0으로 바꿈
data['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED'].map(lambda x: 0 if x > 0 else x)
data.describe()

data_2['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED'].map(lambda x: 0 if x > 0 else x)
data_2.describe()


,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month
count,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000
mean,31456.500,0.435,185043.318,-16020.466,-2192.086,1.000,0.228,0.296,0.086,2.203,-26.272
std,2886.896,0.729,101539.790,4197.673,2352.938,0.000,0.419,0.457,0.280,0.898,16.349
min,26457.000,0.000,27000.000,-25152.000,-15713.000,1.000,0.000,0.000,0.000,1.000,-60.000
25%,28956.750,0.000,121500.000,-19483.250,-3131.000,1.000,0.000,0.000,0.000,2.000,-39.000
50%,31456.500,0.000,157500.000,-15606.000,-1550.000,1.000,0.000,0.000,0.000,2.000,-25.000
75%,33956.250,1.000,225000.000,-12539.000,-413.000,1.000,0.000,1.000,0.000,3.000,-12.000
max,36456.000,5.000,1575000.000,-7489.000,0.000,1.000,1.000,1.000,1.000,7.000,0.000


In [134]:
#음수값 양수로 
feats = ['DAYS_BIRTH', 'begin_month', 'DAYS_EMPLOYED']
for feat in feats:
    data[feat]=np.abs(data[feat])

feats = ['DAYS_BIRTH', 'begin_month', 'DAYS_EMPLOYED']
for feat in feats:
    data_2[feat]=np.abs(data_2[feat])

In [135]:
#DAYS_BIRTH 참고해서 나이 도출
data['age_DAYS_BIRTH'] = round(abs(data['DAYS_BIRTH'])/365,0).astype(np.int32)
data.head(2)

data_2['age_DAYS_BIRTH'] = round(abs(data['DAYS_BIRTH'])/365,0).astype(np.int32)
data_2.head(2)

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,age_DAYS_BIRTH
0,26457,M,Y,N,0,112500.000,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,21990,4709,1,0,1,0,nojob,2.000,60.000,38
1,26458,F,N,Y,0,135000.000,State servant,Higher education,Married,House / apartment,18964,1540,1,0,1,0,Core staff,2.000,36.000,31


In [136]:
# credit 데이터형 변환
data = data.astype({'credit': 'object'})
print(data.dtypes)

index               int64
gender             object
car                object
reality            object
child_num           int64
income_total      float64
income_type        object
edu_type           object
family_type        object
house_type         object
DAYS_BIRTH          int64
DAYS_EMPLOYED       int64
FLAG_MOBIL          int64
work_phone          int64
phone               int64
email               int64
occyp_type         object
family_size       float64
begin_month       float64
credit             object
age_DAYS_BIRTH      int32
dtype: object


In [137]:
#높은 교육수준을 가지고 있는 순으로 4-0 숫자 부여
data['edu_type'] = data['edu_type'].replace(['Higher education', 'Secondary / secondary special', 'Incomplete higher', 'Lower secondary', 'Academic degree' ],[4,3,2,1,0])
data['edu_type'].value_counts()

data_2['edu_type'] = data_2['edu_type'].replace(['Higher education', 'Secondary / secondary special', 'Incomplete higher', 'Lower secondary', 'Academic degree' ],[4,3,2,1,0])
data_2['edu_type'].value_counts()


3    6782
4    2702
2     390
1     117
0       9
Name: edu_type, dtype: int64

In [138]:

y_train=data['credit']
y_train

0       1.000
1       1.000
2       2.000
3       0.000
4       2.000
         ... 
26452   1.000
26453   2.000
26454   2.000
26455   2.000
26456   2.000
Name: credit, Length: 26457, dtype: object

In [139]:
x_train=data.drop(['credit'],axis=1)
x_train


,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,age_DAYS_BIRTH
0,0,F,N,N,0,202500.000,Commercial associate,4,Married,Municipal apartment,13899,4709,1,0,0,0,nojob,2.000,6.000,38
1,1,F,N,Y,1,247500.000,Commercial associate,3,Civil marriage,House / apartment,11380,1540,1,0,0,1,Laborers,3.000,5.000,31
2,2,M,Y,Y,0,450000.000,Working,4,Married,House / apartment,19087,4434,1,0,1,0,Managers,2.000,22.000,52
3,3,F,N,Y,0,202500.000,Commercial associate,3,Married,House / apartment,15088,2092,1,0,1,0,Sales staff,2.000,37.000,41
4,4,F,Y,Y,0,157500.000,State servant,4,Married,House / apartment,15037,2105,1,0,0,0,Managers,2.000,26.000,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,F,N,N,2,225000.000,State servant,3,Married,House / apartment,12079,1984,1,0,0,0,Core staff,4.000,2.000,33
26453,26453,F,N,Y,1,180000.000,Working,4,Separated,House / apartment,15291,2475,1,0,0,0,nojob,2.000,47.000,42
26454,26454,F,Y,N,0,292500.000,Working,3,Civil marriage,With parents,10082,2015,1,0,0,0,Core staff,2.000,25.000,28
26455,26455,M,N,Y,0,171000.000,Working,2,Single / not married,House / apartment,10145,107,1,0,0,0,Laborers,1.000,59.000,28


In [140]:
x_test=data_2

In [141]:
#1 수익을 부양수로 나누기
f = pd.Series(x_train['income_total']/x_train['family_size'], name = '수익/가족수') ; f
x_train=pd.concat([x_train, f], axis =1) ;x_train

f = pd.Series(x_test['income_total']/x_test['family_size'], name = '수익/가족수') ; f
x_test=pd.concat([x_test, f], axis =1) ;x_test


,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,...,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,age_DAYS_BIRTH,수익/가족수
0,26457,M,Y,N,0,112500.000,Pensioner,3,Civil marriage,House / apartment,...,4709,1,0,1,0,nojob,2.000,60.000,38,56250.000
1,26458,F,N,Y,0,135000.000,State servant,4,Married,House / apartment,...,1540,1,0,1,0,Core staff,2.000,36.000,31,67500.000
2,26459,F,N,Y,0,69372.000,Working,3,Married,House / apartment,...,4434,1,1,1,0,Laborers,2.000,40.000,52,34686.000
3,26460,M,Y,N,0,112500.000,Commercial associate,3,Married,House / apartment,...,2092,1,1,0,0,Drivers,2.000,41.000,41,56250.000
4,26461,F,Y,Y,0,225000.000,State servant,4,Married,House / apartment,...,2105,1,1,0,0,Managers,2.000,8.000,41,112500.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,36452,F,Y,Y,0,202500.000,Working,2,Married,House / apartment,...,0,1,1,1,0,Accountants,2.000,19.000,62,101250.000
9996,36453,M,Y,Y,0,202500.000,Working,3,Civil marriage,House / apartment,...,596,1,1,0,0,Laborers,2.000,34.000,55,101250.000
9997,36454,F,N,Y,0,292500.000,Working,3,Married,House / apartment,...,5554,1,0,0,0,Medicine staff,2.000,55.000,58,146250.000
9998,36455,F,Y,N,0,180000.000,Commercial associate,3,Married,House / apartment,...,3170,1,0,1,0,nojob,2.000,33.000,28,90000.000


In [142]:
#2 총수입을 약 31만씩 5그룹으로
cut_bins = [0, 337000, 647000,957000, 1267000, 1577000 ]

f = pd.cut(x_train['income_total'],bins = cut_bins, labels=["0","1","2","3",'4']).astype('int'); f
x_train=pd.concat([x_train, f], axis =1) ;x_train

cut_bins = [0, 337000, 647000,957000, 1267000, 1577000 ]

f = pd.cut(x_test['income_total'],bins = cut_bins, labels=["0","1","2","3",'4']).astype('int'); f
x_test=pd.concat([x_test, f], axis =1) ;x_test

x_train.columns.values[-1]='총수입등급'
x_test.columns.values[-1]='총수입등급'

In [143]:
#3 교육등급+총수입등급
f = pd.Series(x_train['edu_type']+x_train['총수입등급'],name = '교육+총수입등급');f
x_train=pd.concat([x_train, f], axis =1) ;x_train

f = pd.Series(x_test['edu_type']+x_test['총수입등급'],name = '교육+총수입등급');f
x_test=pd.concat([x_test, f], axis =1) ;x_test


,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,...,work_phone,phone,email,occyp_type,family_size,begin_month,age_DAYS_BIRTH,수익/가족수,총수입등급,교육+총수입등급
0,26457,M,Y,N,0,112500.000,Pensioner,3,Civil marriage,House / apartment,...,0,1,0,nojob,2.000,60.000,38,56250.000,0,3
1,26458,F,N,Y,0,135000.000,State servant,4,Married,House / apartment,...,0,1,0,Core staff,2.000,36.000,31,67500.000,0,4
2,26459,F,N,Y,0,69372.000,Working,3,Married,House / apartment,...,1,1,0,Laborers,2.000,40.000,52,34686.000,0,3
3,26460,M,Y,N,0,112500.000,Commercial associate,3,Married,House / apartment,...,1,0,0,Drivers,2.000,41.000,41,56250.000,0,3
4,26461,F,Y,Y,0,225000.000,State servant,4,Married,House / apartment,...,1,0,0,Managers,2.000,8.000,41,112500.000,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,36452,F,Y,Y,0,202500.000,Working,2,Married,House / apartment,...,1,1,0,Accountants,2.000,19.000,62,101250.000,0,2
9996,36453,M,Y,Y,0,202500.000,Working,3,Civil marriage,House / apartment,...,1,0,0,Laborers,2.000,34.000,55,101250.000,0,3
9997,36454,F,N,Y,0,292500.000,Working,3,Married,House / apartment,...,0,0,0,Medicine staff,2.000,55.000,58,146250.000,0,3
9998,36455,F,Y,N,0,180000.000,Commercial associate,3,Married,House / apartment,...,0,1,0,nojob,2.000,33.000,28,90000.000,0,3


In [144]:
#4 차+부동산수
f = pd.Series(x_train['car']+x_train['reality'],name = '차+부동산수');f
x_train=pd.concat([x_train, f], axis =1) ;x_train

f = pd.Series(x_test['car']+x_test['reality'],name = '차+부동산수');f
x_test=pd.concat([x_test, f], axis =1) ;x_test

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,...,phone,email,occyp_type,family_size,begin_month,age_DAYS_BIRTH,수익/가족수,총수입등급,교육+총수입등급,차+부동산수
0,26457,M,Y,N,0,112500.000,Pensioner,3,Civil marriage,House / apartment,...,1,0,nojob,2.000,60.000,38,56250.000,0,3,YN
1,26458,F,N,Y,0,135000.000,State servant,4,Married,House / apartment,...,1,0,Core staff,2.000,36.000,31,67500.000,0,4,NY
2,26459,F,N,Y,0,69372.000,Working,3,Married,House / apartment,...,1,0,Laborers,2.000,40.000,52,34686.000,0,3,NY
3,26460,M,Y,N,0,112500.000,Commercial associate,3,Married,House / apartment,...,0,0,Drivers,2.000,41.000,41,56250.000,0,3,YN
4,26461,F,Y,Y,0,225000.000,State servant,4,Married,House / apartment,...,0,0,Managers,2.000,8.000,41,112500.000,0,4,YY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,36452,F,Y,Y,0,202500.000,Working,2,Married,House / apartment,...,1,0,Accountants,2.000,19.000,62,101250.000,0,2,YY
9996,36453,M,Y,Y,0,202500.000,Working,3,Civil marriage,House / apartment,...,0,0,Laborers,2.000,34.000,55,101250.000,0,3,YY
9997,36454,F,N,Y,0,292500.000,Working,3,Married,House / apartment,...,0,0,Medicine staff,2.000,55.000,58,146250.000,0,3,NY
9998,36455,F,Y,N,0,180000.000,Commercial associate,3,Married,House / apartment,...,1,0,nojob,2.000,33.000,28,90000.000,0,3,YN


In [145]:
std_scaler = StandardScaler()

In [146]:
#원핫인코딩
feature_x_train = x_train[['gender','car','reality','income_type','edu_type','family_type','house_type','occyp_type']]
x_train_one = pd.get_dummies(feature_x_train)
x_train_one


feature_x_test = x_test[['gender','car','reality','income_type','edu_type','family_type','house_type','occyp_type']]
x_test_one = pd.get_dummies(feature_x_test)
x_test_one


,edu_type,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,...,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_nojob
0,3,0,1,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,4,1,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,1,0,0,1,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,3,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,3,1,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9998,3,1,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [147]:
# 지금은 트레인과 테스트에서 범주형 데이터를 제외하는 작업
x_train = x_train.drop(['gender','car','reality','income_type','edu_type','family_type','house_type','occyp_type','index','차+부동산수'],axis=1) ; x_train

x_test = x_test.drop(['gender','car','reality','income_type','edu_type','family_type','house_type','occyp_type','index','차+부동산수'],axis=1) ; x_test


,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,age_DAYS_BIRTH,수익/가족수,총수입등급,교육+총수입등급
0,0,112500.000,21990,4709,1,0,1,0,2.000,60.000,38,56250.000,0,3
1,0,135000.000,18964,1540,1,0,1,0,2.000,36.000,31,67500.000,0,4
2,0,69372.000,15887,4434,1,1,1,0,2.000,40.000,52,34686.000,0,3
3,0,112500.000,19270,2092,1,1,0,0,2.000,41.000,41,56250.000,0,3
4,0,225000.000,17822,2105,1,1,0,0,2.000,8.000,41,112500.000,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,202500.000,18593,0,1,1,1,0,2.000,19.000,62,101250.000,0,2
9996,0,202500.000,10886,596,1,1,0,0,2.000,34.000,55,101250.000,0,3
9997,0,292500.000,21016,5554,1,0,0,0,2.000,55.000,58,146250.000,0,3
9998,0,180000.000,16541,3170,1,0,1,0,2.000,33.000,28,90000.000,0,3


In [148]:
std_scaler.fit(x_train)
std_scaler.fit(x_test)

x_train_scaled = std_scaler.transform(x_train)
x_train_scaled 

x_test_scaled = std_scaler.transform(x_test)
x_test_scaled 

array([[-0.59624306, -0.71446815,  1.42217659, ..., -0.60186622,
        -0.25876031, -0.41802889],
       [-0.59624306, -0.49286906,  0.70126493, ..., -0.44409537,
        -0.25876031,  1.05910499],
       [-0.59624306, -1.13922928, -0.03179692, ..., -0.90428138,
        -0.25876031, -0.41802889],
       ...,
       [-0.59624306,  1.05832457,  1.19013166, ...,  0.66030056,
        -0.25876031, -0.41802889],
       [-0.59624306, -0.04967088,  0.12401148, ..., -0.12855368,
        -0.25876031, -0.41802889],
       [-0.59624306,  0.83672548, -1.63586109, ...,  0.50252971,
        -0.25876031,  1.05910499]])

In [149]:
from catboost import CatBoostClassifier
model = CatBoostClassifier()
model.fit(x_train,
          y_train,
          verbose=False)

pred =model.predict(x_test) ; pred

probability = model.predict_proba( x_test )
print( probability )


[[0.18564863 0.14446411 0.66988726]
 [0.13716992 0.235141   0.62768908]
 [0.08937869 0.16884528 0.74177603]
 ...
 [0.0506251  0.14327541 0.80609949]
 [0.11543205 0.19676744 0.68780051]
 [0.0514385  0.16796721 0.78059429]]


In [150]:
submission=pd.read_csv('sample_submission.csv')
submission.loc[:,1:]=probability
submission

,index,0,1,2
0,26457,0.186,0.144,0.670
1,26458,0.137,0.235,0.628
2,26459,0.089,0.169,0.742
3,26460,0.097,0.209,0.694
4,26461,0.086,0.165,0.749
...,...,...,...,...
9995,36452,0.118,0.124,0.758
9996,36453,0.138,0.187,0.675
9997,36454,0.051,0.143,0.806
9998,36455,0.115,0.197,0.688


In [151]:
submission=submission.rename(columns={0:'0',1:'1',2:'2'})

In [152]:
submission.to_csv('sample_submission.csv',index=False)


In [153]:
submission.shape[1]

4

In [154]:
submission.shape[0]

10000